# DATA LOADING

#### After initial analysis found that data contains unnecessary spaces in the json files

In [282]:
# Function for loading data that solves discrepencies 
import json
import gzip
import pandas as pd
from datetime import datetime

# Helper function to load gzipped JSON data
def load_json_gz(file_path):
    # Open the gzipped file and read lines, starting from the first valid JSON entry
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        data = []
        for line in file:
            # Attempt to parse each line as JSON
            try:
                json_object = json.loads(line)
                data.append(json_object)
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue
    return data
# Helper function to convert timestamp to datetime
def convert_timestamp_safe(timestamp):
    if timestamp is None:
        return None
    return datetime.utcfromtimestamp(timestamp / 1000)

#### Please replace with your files data path in the below variables

In [283]:
from pandas import json_normalize
# File path for users
file_path1 = '/Users/vishnusaibhadramraju/Desktop/ASSESSMENT/users.json.gz'
# file path for brands
file_path2 = '/Users/vishnusaibhadramraju/Desktop/ASSESSMENT/brands.json.gz'
#file path for receipts
file_path3 = '/Users/vishnusaibhadramraju/Desktop/ASSESSMENT/receipts.json.gz'


data = load_json_gz(file_path1)
data1 = load_json_gz(file_path2)
# Loading the users and brands tables data
users_df = pd.json_normalize(data)
brands_df = pd.json_normalize(data1)

# The receipts table contains nearly 1119 json objects and table is much more unstructured than the above two tables
# Load the JSON data from the receipts file
receipts_data = load_json_gz(file_path3)

# Display the first JSON object to see its structure
receipts_sample = receipts_data[0]

#### After inspection found out receipts contains two tables in it

In [284]:
# Initialize lists to store data for receipts and receipt items
receipts = []
receipt_items = []

# Extract data for receipts and receipt items
for record in receipts_data:
    receipt = {
        "receipt_id": record['_id']['$oid'],
        "user_id": record['userId'],
        "create_date": convert_timestamp_safe(record['createDate']['$date']),
        "date_scanned": convert_timestamp_safe(record['dateScanned']['$date']),
        "finished_date": convert_timestamp_safe(record.get('finishedDate', {}).get('$date')),
        "points_awarded_date": convert_timestamp_safe(record.get('pointsAwardedDate', {}).get('$date')),
        "points_earned": float(record.get('pointsEarned', 0)),
        "purchase_date": convert_timestamp_safe(record.get('purchaseDate', {}).get('$date')),
        "purchased_item_count": record.get('purchasedItemCount', 0),
        "rewards_receipt_status": record.get('rewardsReceiptStatus', None),
        "total_spent": float(record.get('totalSpent', 0))
    }
    receipts.append(receipt)
    
    for item in record.get('rewardsReceiptItemList', []):
        receipt_item = {
            "receipt_id": receipt["receipt_id"],
            "barcode": item.get('barcode', None),
            "description": item.get('description', None),
            "final_price": float(item.get('finalPrice', 0)),
            "item_price": float(item.get('itemPrice', 0)),
            "quantity_purchased": int(item.get('quantityPurchased', 0)),
            "points_earned": float(item.get('pointsEarned', 0)),
            "points_not_awarded_reason": item.get('pointsNotAwardedReason', None),
            "rewards_group": item.get('rewardsGroup', None),
            "rewards_product_partner_id": item.get('rewardsProductPartnerId', None)
        }
        receipt_items.append(receipt_item)

# Convert lists to DataFrames
receipts_df = pd.DataFrame(receipts)
receipt_items_df = pd.DataFrame(receipt_items)


#### Data view of each table

In [285]:
users_df.head()

,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
3,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12


In [286]:
brands_df.head()

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [287]:
receipts_df.head()

,receipt_id,user_id,create_date,date_scanned,finished_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,total_spent
0,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5,FINISHED,26.0
1,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2,FINISHED,11.0
2,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,NaT,5.0,2021-01-03 00:00:00,1,REJECTED,10.0
3,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4,FINISHED,28.0
4,5ff1e1d20a7214ada1000561,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2,FINISHED,1.0


In [288]:
receipt_items_df.head()

,receipt_id,barcode,description,final_price,item_price,quantity_purchased,points_earned,points_not_awarded_reason,rewards_group,rewards_product_partner_id
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.0,26.0,5,0.0,None,None,None
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1.0,1.0,1,0.0,None,None,None
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.0,10.0,1,0.0,Action not allowed for user and CPG,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba
3,5ff1e1f10a720f052300057a,None,None,0.0,0.0,0,0.0,None,None,None
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.0,28.0,4,0.0,None,None,None


# Data Cleaning

In [289]:
#Users data 
users_df.columns

Index(['active', 'role', 'signUpSource', 'state', '_id.$oid',
       'createdDate.$date', 'lastLogin.$date'],
      dtype='object')

In [290]:
# Clean up DataFrame column names
users_df.columns = users_df.columns.str.replace('.$', '')

In [291]:
users_df = users_df.rename(columns={
    '_idoid': 'user_id',
    'createdDatedate': 'created_date',
    'lastLogindate': 'last_login',
    'role': 'role',
    'signUpSource': 'signup_source',
    'state': 'state'
})

In [292]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   active         494 non-null    bool   
 1   role           494 non-null    object 
 2   signup_source  446 non-null    object 
 3   state          438 non-null    object 
 4   user_id        494 non-null    object 
 5   created_date   494 non-null    int64  
 6   last_login     432 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [293]:
users_df['created_date'] = pd.to_datetime(users_df['created_date'], unit='ms',errors='coerce')
users_df['last_login'] = pd.to_datetime(users_df['last_login'], unit='ms',errors='coerce')# converting NaT to Null since postgres wont accept NaT


In [294]:
# Converting NaT values in timestamp fields to NULL since postgresql only accepts NULL
users_df['last_login'].fillna('NULL',inplace = True)

/var/folders/tg/vkj956ss1b108j97r7gcts7r0000gn/T/ipykernel_50642/2499856600.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'NULL' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  users_df['last_login'].fillna('NULL',inplace = True)


In [295]:
#Brands tables
brands_df.columns

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', '_id.$oid',
       'cpg.$id.$oid', 'cpg.$ref', 'brandCode'],
      dtype='object')

In [296]:
# Clean up DataFrame column names
brands_df.columns = brands_df.columns.str.replace('.$', '')

In [297]:
brands_df = brands_df.rename(columns={
    'barcode': 'barcode',
    'category': 'category',
    'categoryCode': 'category_code',
    'name': 'brand_name',
    'topBrand': 'top_brand',
    '_idoid': 'brand_id',
    'cpgidoid': 'cpg_id',
    'cpg.$ref': 'cpg_ref',
    'brandCode': 'brand_code'
})

In [298]:
brands_df['top_brand'].fillna('NULL', inplace = True)

## DATA QUALITY CHECKS 

### Missing Values

In [299]:
# Missing Values
users_df.isnull().sum()

active            0
role              0
signup_source    48
state            56
user_id           0
created_date      0
last_login        0
dtype: int64

In [300]:
#Lets Inspect 
users_df.shape

(494, 7)

### Uniqueness

In [301]:
users_df.nunique()

active             2
role               2
signup_source      2
state              8
user_id          212
created_date     212
last_login       173
dtype: int64

### Duplicates 

In [302]:
#  lets check uniqueness
users_df.user_id.value_counts()

user_id
54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5ff5d15aeb7c7d12096d91a2    18
5fa41775898c7a11a6bcef3e    18
59c124bae4b0299e55b0f330    18
                            ..
6004a965e257124ec6b9a39f     1
600746fd6e64691717e8cfb5     1
60074b996e64691717e8f11a     1
60074246325c8a12289e22a0     1
60088e5d633aab121bb8e5cf     1
Name: count, Length: 212, dtype: int64

In [303]:
# lets clear duplicates 
users_df.drop_duplicates(subset='user_id', inplace = True)

In [304]:
users_df.nunique()

active             2
role               2
signup_source      2
state              8
user_id          212
created_date     212
last_login       173
dtype: int64

In [305]:
users_df.shape

(212, 7)

### Solution for filling NaN
### We can drop since me may loose the unique user
### We can fill with backfill or ffill or most frequent, im using most frequent fill

In [306]:
#We can fill with backfill or ffill or most frequent but i want to keept it like this

In [307]:
most_frequent_signup = users_df['signup_source'].mode()[0]
most_frequent_state = users_df['state'].mode()[0]

In [308]:
users_df.signup_source.fillna(most_frequent_signup, inplace = True)
users_df.state.fillna(most_frequent_state, inplace = True)

In [309]:
users_df.isnull().sum()

active           0
role             0
signup_source    0
state            0
user_id          0
created_date     0
last_login       0
dtype: int64

In [310]:
brands_df.shape

(1167, 9)

In [311]:
brands_df.isnull().sum()

barcode            0
category         155
category_code    650
brand_name         0
top_brand          0
brand_id           0
cpg_id             0
cpgref             0
brand_code       234
dtype: int64

In [312]:
brands_df.nunique()

barcode          1160
category           23
category_code      14
brand_name       1156
top_brand           3
brand_id         1167
cpg_id            196
cpgref              2
brand_code        897
dtype: int64

In [313]:
# We found that we have duplicated brands name and barcode lets fix this
brand_name_counts = brands_df.groupby('brand_name')['brand_id'].count().reset_index()

In [314]:
brand_name_counts

,brand_name,brand_id
0,.,1
1,1915 Bolthouse Farms,1
2,1_KRAFT Hockeyville,1
3,7 up,1
4,A&W Rootbeer,1
...,...,...
1151,test brand @1612366146091,1
1152,test brand @1612366146133,1
1153,test brand @1612366146176,1
1154,test brand @1612366146827,1


In [315]:
brands_with_multiple_ids = brand_name_counts[brand_name_counts['brand_id'] > 1]
brands_with_multiple_ids

,brand_name,brand_id
73,Baken-Ets,2
129,Caleb's Kola,2
215,Diabetic Living Magazine,2
223,Dippin Dots® Cereal,2
313,Health Magazine,2
333,Huggies,2
335,I CAN'T BELIEVE IT'S NOT BUTTER!,2
504,ONE A DAY® WOMENS,2
564,Pull-Ups,2
627,Sierra Mist,2


In [316]:
# Lets dive deep
brands_df.loc[brands_df['brand_name'].isin(brands_with_multiple_ids['brand_name'])].sort_values(by='brand_name')

,barcode,category,category_code,brand_name,top_brand,brand_id,cpg_id,cpgref,brand_code
848,511111701781,Snacks,NaN,Baken-Ets,True,585a961fe4b03e62d1ce0e76,5332f5fbe4b03c9a25efd0ba,Cogs,BAKEN-ETS
574,511111605546,Snacks,NaN,Baken-Ets,NULL,5d9d08d1a60b87376833e348,5332f5fbe4b03c9a25efd0ba,Cogs,BAKEN ETS
140,511111000518,Beverages,NaN,Caleb's Kola,False,5a4d23dae4b0bcb2c74ea77e,5332f5fbe4b03c9a25efd0ba,Cogs,CALEB'S KOLA
740,511111004912,Snacks,NaN,Caleb's Kola,NULL,5d601d74a3a018514994f422,53e10d6368abd3c7065097cc,Cogs,CALEBS KOLA
1007,511111205227,NaN,NaN,Diabetic Living Magazine,NULL,5d658ffa6d5f3b23d1bc7914,53e10d6368abd3c7065097cc,Cogs,NaN
1006,511111805298,Magazines,NaN,Diabetic Living Magazine,NULL,5d66d597a3a018093ab34726,5d5d4fd16d5f3b23d1bc7905,Cogs,511111805298
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NULL,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs,DIPPIN DOTS CEREAL
1081,511111206330,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NULL,5dc2d9d4a60b873d6b0666d2,5332f5f3e4b03c9a25efd0ae,Cogs,DIPPIN DOTS
194,511111605058,Magazines,NaN,Health Magazine,NULL,5d6415d5a3a018514994f429,5d5d4fd16d5f3b23d1bc7905,Cogs,511111605058
596,511111915287,Magazines,MAGAZINES,Health Magazine,NULL,5f298852be37ce7958c5952d,5d66b9dcee7f2d201c7281cd,Cogs,HEALTH


#### We found that are same brand name with same data but created with different brand_id, we can remove the duplicated ones

In [317]:
brands_df.drop_duplicates(subset= 'brand_name', keep = 'first', inplace = True)

In [318]:
brands_df.shape

(1156, 9)

In [319]:
brands_df.nunique()

barcode          1149
category           23
category_code      13
brand_name       1156
top_brand           3
brand_id         1156
cpg_id            195
cpgref              2
brand_code        888
dtype: int64

In [320]:
# We also found duplicated barcodes lets inspect

In [321]:
brands_df.isnull().sum()

barcode            0
category         154
category_code    642
brand_name         0
top_brand          0
brand_id           0
cpg_id             0
cpgref             0
brand_code       233
dtype: int64

In [322]:
barcodes_count =  pd.DataFrame(brands_df.barcode.value_counts())
barcodes_count

,count
barcode,
511111305125,2
511111605058,2
511111504139,2
511111204923,2
511111004790,2
...,...
511111703334,1
511111705727,1
511111302414,1


In [323]:
barcodes_multiple_id = brands_df.groupby(by='barcode')['brand_id'].count().reset_index()


In [324]:
#lets see unique values in our count column
barcodes_multiple_id.brand_id.unique()

array([1, 2])

In [325]:
barcodes_multiple_id_filter = barcodes_multiple_id.loc[barcodes_multiple_id['brand_id']>1]
barcodes_multiple_id_filter

,barcode,brand_id
46,511111004790,2
281,511111204923,2
397,511111305125,2
617,511111504139,2
623,511111504788,2
740,511111605058,2
846,511111704140,2


In [326]:
brands_df.loc[brands_df['barcode'].isin(barcodes_multiple_id_filter['barcode'])].sort_values(by='barcode')

,barcode,category,category_code,brand_name,top_brand,brand_id,cpg_id,cpgref,brand_code
467,511111004790,Baking,NaN,alexa,True,5c409ab4cd244a3539b84162,55b62995e4b0d8e685c14213,Cogs,ALEXA
1071,511111004790,Condiments & Sauces,NaN,Bitten Dressing,NULL,5cdacd63166eb33eb7ce0fa8,559c2234e4b06aca36af13c6,Cogs,BITTEN
152,511111204923,Grocery,NaN,Brand1,True,5c45f91b87ff3552f950f027,5c45f8b087ff3552f950f026,Cogs,0987654321
536,511111204923,Snacks,NaN,CHESTER'S,NULL,5d6027f46d5f3b23d1bc7906,5332f5fbe4b03c9a25efd0ba,Cogs,CHESTERS
20,511111305125,Baby,NaN,Chris Image Test,NULL,5c4699f387ff3577e203ea29,55b62995e4b0d8e685c14213,Cogs,CHRISIMAGE
651,511111305125,Magazines,NaN,Rachael Ray Everyday,NULL,5d642d65a3a018514994f42d,5d5d4fd16d5f3b23d1bc7905,Cogs,511111305125
129,511111504139,Beverages,NaN,Chris Brand XYZ,NULL,5a7e0604e4b0aedb3b84afd3,55b62995e4b0d8e685c14213,Cogs,CHRISXYZ
299,511111504139,Grocery,NaN,Pace,False,5a8c33f3e4b07f0a2dac8943,5a734034e4b0d58f376be874,Cogs,PACE
9,511111504788,Baking,NaN,test,NULL,5c408e8bcd244a1fdb47aee7,59ba6f1ce4b092b29c167346,Cogs,TEST
412,511111504788,Condiments & Sauces,NaN,The Pioneer Woman,NULL,5ccb2ece166eb31bbbadccbe,559c2234e4b06aca36af13c6,Cogs,PIONEER WOMAN


### Possible assumptions
### Here we can make an assumption that a brand_name can have multiple barcodes
### Another assumption a barcode can be used to same category item of different brands_name
### My assumption according to table is that these wrongly populated, since we have a unqiue brand name has unique brand_id and each brand_name must have its own barcode 

### So leaving them without any transformation, since i generated a question for stake holders based on this 

In [327]:
receipts_df.head()

,receipt_id,user_id,create_date,date_scanned,finished_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,total_spent
0,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5,FINISHED,26.0
1,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2,FINISHED,11.0
2,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,NaT,5.0,2021-01-03 00:00:00,1,REJECTED,10.0
3,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4,FINISHED,28.0
4,5ff1e1d20a7214ada1000561,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2,FINISHED,1.0


In [328]:
receipts_df.shape

(1119, 11)

In [329]:
receipts_df.nunique()

receipt_id                1119
user_id                    258
create_date               1107
date_scanned              1107
finished_date              553
points_awarded_date        523
points_earned              119
purchase_date              358
purchased_item_count        50
rewards_receipt_status       5
total_spent                 94
dtype: int64

In [330]:
receipts_df.isnull().sum()

receipt_id                  0
user_id                     0
create_date                 0
date_scanned                0
finished_date             551
points_awarded_date       582
points_earned               0
purchase_date             448
purchased_item_count        0
rewards_receipt_status      0
total_spent                 0
dtype: int64

### Here we found that we have 1119 unique receipts, and null values in the columns finished date, points_awarded_date, purchase_date


### Lets see here we can fill date_scanned values into finished_date

In [331]:
# Compare the two columns
comparison = receipts_df['date_scanned'] == receipts_df['finished_date']
not_equal = receipts_df['date_scanned'] != receipts_df['finished_date']

# Count the number of rows where the values are the same
count_same_values = comparison.sum()
not_equal_values = not_equal.sum()

In [332]:
count_same_values

165

In [333]:
not_equal_values

954

### Here we found that we can fill since there a huge difference between them, we can drop or fill them since we dont know their usuability Yet

### Let check purchase date column whether we can fill with date_scanned since purchase date mostly associated with scanned_date

In [334]:
comparison_purchase_date = receipts_df['date_scanned'] == receipts_df['purchase_date']
#not_equal = receipts_df['date_scanned'] != receipts_df['finished_date']

In [335]:

count_same_values_purchase = comparison_purchase_date.sum()

In [336]:
count_same_values_purchase

30

### Seems ike we can't fill with date_scanned since we have little or no equal values

### We can also check the above columns with created_date but we have little or no information their relationship between them 

In [337]:
#Lets Check 

In [338]:
comparison_created_date = receipts_df['create_date'] == receipts_df['purchase_date']
comparison_created_date.sum()
# No relation

30

### I'm leaving them with out filling them or removing them without further information 

### Data missing in the users table 
### If we see unique users in receipts is 258 and in users table 212, theres is data ineqality or missing data 

In [339]:
# Lets solve 

In [340]:
count_of_users_in = receipts_df.loc[receipts_df['user_id'].isin(users_df['user_id'])]

In [341]:
count_of_users_in.nunique()

receipt_id                971
user_id                   141
create_date               961
date_scanned              961
finished_date             445
points_awarded_date       407
points_earned              89
purchase_date             293
purchased_item_count       35
rewards_receipt_status      5
total_spent                69
dtype: int64

In [342]:
count_of_users_not_in = receipts_df.loc[~receipts_df['user_id'].isin(users_df['user_id'])]

In [343]:
count_of_users_not_in.nunique()

receipt_id                148
user_id                   117
create_date               146
date_scanned              146
finished_date             108
points_awarded_date       116
points_earned              51
purchase_date              87
purchased_item_count       27
rewards_receipt_status      5
total_spent                49
dtype: int64

### It shows that out of 258 total users only 141 are common in both table and 117 users are not present in users

In [344]:
## Let add the remaining users to Users table

In [345]:
users = pd.concat([users_df,count_of_users_not_in])

In [346]:
users.nunique()

active                      2
role                        2
signup_source               2
state                       8
user_id                   329
created_date              212
last_login                173
receipt_id                148
create_date               146
date_scanned              146
finished_date             108
points_awarded_date       116
points_earned              51
purchase_date              87
purchased_item_count       27
rewards_receipt_status      5
total_spent                49
dtype: int64

In [347]:
users.shape

(360, 17)

In [348]:
users = users[users.columns[:7]]

In [349]:
users

,active,role,signup_source,state,user_id,created_date,last_login
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
2,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
5,True,consumer,Email,WI,5ff1e1e8cfcf6c399c274ad9,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392000
6,True,consumer,Email,WI,5ff1e1b7cfcf6c399c274a5a,2021-01-03 15:24:39.626,2021-01-03 15:24:39.664999936
8,True,consumer,Email,WI,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599000064
...,...,...,...,...,...,...,...
955,NaN,NaN,NaN,NaN,60253861efa6017a44dc6b50,NaT,NaN
956,NaN,NaN,NaN,NaN,60253891b54593795bf69242,NaT,NaN
966,NaN,NaN,NaN,NaN,60253891b54593795bf69242,NaT,NaN
985,NaN,NaN,NaN,NaN,60268c7bb545931ac63683af,NaT,NaN


In [350]:
users.drop_duplicates(subset= 'user_id', inplace = True)

In [351]:
users.nunique()

active             2
role               2
signup_source      2
state              8
user_id          329
created_date     212
last_login       173
dtype: int64

In [352]:
users_df = users

In [353]:
# Let's inspect receipts_item
receipt_items_df.head()

,receipt_id,barcode,description,final_price,item_price,quantity_purchased,points_earned,points_not_awarded_reason,rewards_group,rewards_product_partner_id
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.0,26.0,5,0.0,None,None,None
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1.0,1.0,1,0.0,None,None,None
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.0,10.0,1,0.0,Action not allowed for user and CPG,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba
3,5ff1e1f10a720f052300057a,None,None,0.0,0.0,0,0.0,None,None,None
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.0,28.0,4,0.0,None,None,None


In [354]:
receipt_items_df.shape

(6941, 10)

In [355]:
receipt_items_df.nunique()

receipt_id                     679
barcode                        568
description                   1889
final_price                    823
item_price                     823
quantity_purchased              14
points_earned                  278
points_not_awarded_reason        1
rewards_group                  182
rewards_product_partner_id      16
dtype: int64

In [356]:
receipt_items_df.isnull().sum()

receipt_id                       0
barcode                       3851
description                    381
final_price                      0
item_price                       0
quantity_purchased               0
points_earned                    0
points_not_awarded_reason     6601
rewards_group                 5210
rewards_product_partner_id    4672
dtype: int64

### After inspection we found that we have nulls in points_not_awarded_reason, rewards_group, rewards_product_partner_id have most nulls values 
### we can drop them but i want to keep them until further information from stakeholders i want to learn about the significance of the columns

In [357]:
receipt_items_df.description.value_counts()

description
ITEM NOT FOUND                                                         173
KLARBRUNN 12PK 12 FL OZ                                                120
HUGGIES SIMPLY CLEAN PREMOISTENED WIPE FRAGRANCE FREE BAG 216 COUNT     92
MILLER LITE 24 PACK 12OZ CAN                                            90
KLEENEX POP UP RECTANGLE BOX FACIAL TISSUE 2 PLY 8PK 160 CT             87
                                                                      ... 
434 BETTYCRKRB                                                           1
433 KROGERSOUR                                                           1
430 DOLESALAD K                                                          1
424 RAGU PASTAS                                                          1
Berry Strawberry Conventional, 16 Ounce                                  1
Name: count, Length: 1889, dtype: int64

### I can fill description with fillna methods, but right now im just keeping it as it is since we dont have any use of the column for the posted questions

# Loading Data into POSTGRESQL for sql queries

In [358]:
# since Postgres doesnt accept 'NaT' nulls we need convert into NUlls
users_df.fillna('NULL', inplace = True)

/var/folders/tg/vkj956ss1b108j97r7gcts7r0000gn/T/ipykernel_50642/998433314.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'NULL' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  users_df.fillna('NULL', inplace = True)


In [359]:
# Users table

import psycopg2
# Connection parameters
conn_params = {
    'dbname': 'Sales',
    'user': 'postgres',
    'password': '2000',
    'host': 'localhost',
    'port': 5432
}
# Connect to PostgreSQL
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# Create an insert query
insert_query = """
INSERT INTO users (user_id, active, created_date, last_login, role, signup_source, state)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (user_id) DO NOTHING;
"""

# Insert data into the PostgreSQL table
for i, row in users_df.iterrows():
    cur.execute(insert_query, (
        row['user_id'], 
        row['active'], 
        row['created_date'], 
        row['last_login'], 
        row['role'], 
        row['signup_source'], 
        row['state']
    ))

# Commit the transaction and close the connection
conn.commit()
cur.close()
conn.close()


In [265]:
# Receipts  tables

import psycopg2
# Connection parameters
conn_params = {
    'dbname': 'Sales',
    'user': 'postgres',
    'password': '2000',
    'host': 'localhost',
    'port': 5432
}
# Connect to PostgreSQL
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# Create an insert query
insert_query = """
INSERT INTO receipts (receipt_id, user_id, create_date, date_scanned, finished_date, points_awarded_date, points_earned, purchase_date, purchased_item_count, rewards_receipt_status, total_spent)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (receipt_id) DO NOTHING;
"""

# Insert data into the PostgreSQL table
for i, row in receipts_df.iterrows():
    cur.execute(insert_query, (
        row['receipt_id'], 
        row['user_id'], 
        row['create_date'], 
        row['date_scanned'], 
        row['finished_date'], 
        row['points_awarded_date'], 
        row['points_earned'],
        row['purchase_date'],
        row['purchased_item_count'],
        row['rewards_receipt_status'],
        row['total_spent']
    ))

# Commit the transaction and close the connection
conn.commit()
cur.close()
conn.close()


In [266]:
# receipt_items_df
import psycopg2
# Connection parameters
conn_params = {
    'dbname': 'Sales',
    'user': 'postgres',
    'password': '2000',
    'host': 'localhost',
    'port': 5432
}
# Connect to PostgreSQL
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()


# Create an insert query
insert_query = """
INSERT INTO receipt_items(receipt_id, barcode, description, final_price, item_price, quantity_purchased, points_earned, points_not_awarded_reason,rewards_group,rewards_product_partner_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Insert data into the PostgreSQL table
for i, row in receipt_items_df.iterrows():
    cur.execute(insert_query, (
        row['receipt_id'], 
        row['barcode'], 
        row['description'], 
        row['final_price'], 
        row['item_price'], 
        row['quantity_purchased'], 
        row['points_earned'],
        row['points_not_awarded_reason'],
        row['rewards_group'],
        row['rewards_product_partner_id']
    ))

# Commit the transaction and close the connection
conn.commit()
cur.close()
conn.close()


In [269]:
# brands
import psycopg2
# Connection parameters
conn_params = {
    'dbname': 'Sales',
    'user': 'postgres',
    'password': '2000',
    'host': 'localhost',
    'port': 5432
}
# Connect to PostgreSQL
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# Create an insert query
insert_query = """
INSERT INTO brands (barcode, category, category_code, brand_name, top_brand, brand_id, cpg_id, cpgref, brand_code)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (brand_id) DO NOTHING;
"""

# Insert data into the PostgreSQL table
for i, row in brands_df.iterrows():
    cur.execute(insert_query, (
        row['barcode'], 
        row['category'], 
        row['category_code'], 
        row['brand_name'], 
        row['top_brand'], 
        row['brand_id'], 
        row['cpg_id'],
        row['cpgref'],
        row['brand_code']
    ))

# Commit the transaction and close the connection
conn.commit()
cur.close()
conn.close()


In [270]:
receipts_df

,receipt_id,user_id,create_date,date_scanned,finished_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,total_spent
0,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5,FINISHED,26.00
1,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2,FINISHED,11.00
2,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NULL,NULL,5.0,2021-01-03 00:00:00,1,REJECTED,10.00
3,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4,FINISHED,28.00
4,5ff1e1d20a7214ada1000561,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2,FINISHED,1.00
...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,5fc961c3b8cfca11a077dd33,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NULL,NULL,25.0,2020-08-17 00:00:00,2,REJECTED,34.96
1115,603d0b710a720fde1000042a,5fc961c3b8cfca11a077dd33,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NULL,NULL,0.0,NULL,0,SUBMITTED,0.00
1116,603cf5290a720fde10000413,5fc961c3b8cfca11a077dd33,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NULL,NULL,0.0,NULL,0,SUBMITTED,0.00
1117,603ce7100a7217c72c000405,5fc961c3b8cfca11a077dd33,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NULL,NULL,25.0,2020-08-17 00:00:00,2,REJECTED,34.96


In [276]:
receipts_df.tail()

,receipt_id,user_id,create_date,date_scanned,finished_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,total_spent
1114,603cc0630a720fde100003e6,5fc961c3b8cfca11a077dd33,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NULL,NULL,25.0,2020-08-17 00:00:00,2,REJECTED,34.96
1115,603d0b710a720fde1000042a,5fc961c3b8cfca11a077dd33,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NULL,NULL,0.0,NULL,0,SUBMITTED,0.00
1116,603cf5290a720fde10000413,5fc961c3b8cfca11a077dd33,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NULL,NULL,0.0,NULL,0,SUBMITTED,0.00
1117,603ce7100a7217c72c000405,5fc961c3b8cfca11a077dd33,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NULL,NULL,25.0,2020-08-17 00:00:00,2,REJECTED,34.96
1118,603c4fea0a7217c72c000389,5fc961c3b8cfca11a077dd33,2021-03-01 02:22:34.962,2021-03-01 02:22:34.962,NULL,NULL,0.0,NULL,0,SUBMITTED,0.00


,active,role,signup_source,state,user_id,created_date,last_login
925,Null,Null,Null,Null,6024291f58b989121e6bad5f,Null,Null
948,Null,Null,Null,Null,60253861efa6017a44dc6b50,Null,Null
956,Null,Null,Null,Null,60253891b54593795bf69242,Null,Null
985,Null,Null,Null,Null,60268c7bb545931ac63683af,Null,Null
990,Null,Null,Null,Null,60268c78efa6011bb151077d,Null,Null
